In [2]:
# load data into dataframe

import numpy as np
import pandas as pd

ui_data = pd.read_json('../data/australian_users_items_fixed.json')
ur_data = pd.read_json('../data/australian_user_reviews_fixed.json')


In [92]:
# creating output format

ratings = {'user_id': [],
           'item_id': [],
           'rating': []
          } 


#rating criteria:

'''
Playtime:
>= 30 +2,
> 300 +1,
> 1000 +1

Recommend:
recommend +1
not recommend -1
'''

'\nPlaytime:\n>= 30 +2,\n> 300 +1,\n> 1000 +1\n\nRecommend:\nrecommend +1\nnot recommend -1\n'

In [68]:
# function for extracting item_id

def extract_item_ids(reviews):
    item_ids = []
    for i in range(len(reviews)):
        item_ids.append(reviews[0][i]['item_id'])
    return item_ids

In [93]:
# compute rating for each user-item pair

for u in range(len(ui_data)):
    
    user = ui_data.iloc[u]
    
    for num in range(user['items_count']):
        
        rating = 0
        ratings['user_id'].append(user['user_id'])
        ratings['item_id'].append(user['items'][num]['item_id'])
        
        
        if 30 <= user['items'][num]['playtime_forever'] < 300:
            rating += 2
        elif 300 <= user['items'][num]['playtime_forever'] < 1000:
            rating += 3
        elif 1000 <= user['items'][num]['playtime_forever']:
            rating += 4
        else:
            rating += 0
        
        if user['user_id'] in ur_data['user_id']:
            
            reviews = ur_data.loc[ur_data['user_id'] == user['user_id']]['reviews']
            
            if user['items'][num]['item_id'] in extract_item_ids(reviews):
            
                review = reviews['item_id' == user['items'][num]['item_id']]
                
                if review['recommend'] == True:
                    rating += 1
                elif review['recommend'] == False:
                    rating -= 1
                else:
                    ratings['rating'].append(rating)
                    continue
            else:
                ratings['rating'].append(rating)
                continue
        else:
            ratings['rating'].append(rating)
            continue

In [94]:
# output the result to a .csv file

output_df = pd.DataFrame(ratings, columns = ['user_id','item_id','rating']) # forming the dataframe

output_df.to_csv(r'/Users/brian80433/Desktop/256Project/data/output.csv', index = False, header=True) # output file



In [97]:
ratings_df  = pd.read_csv('../data/output.csv')

In [98]:
ratings_df

,user_id,item_id,rating
0,76561197970982479,10,0
1,76561197970982479,20,0
2,76561197970982479,30,0
3,76561197970982479,40,0
4,76561197970982479,50,0
...,...,...,...
5153204,76561198329548331,346330,0
5153205,76561198329548331,373330,0
5153206,76561198329548331,388490,0
5153207,76561198329548331,521570,0


In [100]:
from surprise import BaselineOnly
from surprise import Dataset
from surprise import Reader
from surprise.model_selection.split import train_test_split
from surprise.model_selection import cross_validate, GridSearchCV
import pandas as pd
import numpy as np
import os, io
from surprise import KNNBasic, KNNWithMeans
from surprise import SVDpp
from surprise import SVD
from surprise import accuracy

In [101]:
reader = Reader(rating_scale=(0,5))  #invoke reader instance of surprise library
data=Dataset.load_from_df(ratings_df,reader)

In [102]:
trainingSet, testSet = train_test_split(data, test_size=0.3, train_size=None, random_state=None, shuffle=True)

In [103]:
sim_options = {
    'name': 'pearson', #similarity measure default is MSD
    'user_based': False #user-based CF
}

In [104]:
#KNN
knn = KNNBasic(sim_options=sim_options,k=45,min_k=15) #neighbours=3, other parameters set as above
knn.fit(trainingSet) #fit model to the training set
predictions_knn = knn.test(testSet) #predict for test set values

Computing the pearson similarity matrix...
Done computing similarity matrix.


In [105]:
accuracy.rmse(predictions_knn, verbose=True) 

RMSE: 1.3537


1.3536926005968417

In [106]:
predictions_knn

[Prediction(uid='eyeharp', iid=240, r_ui=3.0, est=2.2738183499416897, details={'actual_k': 44, 'was_impossible': False}),
 Prediction(uid='76561198062196434', iid=34900, r_ui=0.0, est=0.8511446061026726, details={'actual_k': 45, 'was_impossible': False}),
 Prediction(uid='76561198045431856', iid=44690, r_ui=0.0, est=2.0075845456609267, details={'actual_k': 45, 'was_impossible': False}),
 Prediction(uid='dalash', iid=105600, r_ui=0.0, est=1.4978928451428692, details={'actual_k': 31, 'was_impossible': False}),
 Prediction(uid='76561198071247265', iid=100, r_ui=0.0, est=1.4290350588787124, details={'was_impossible': True, 'reason': 'Not enough neighbors.'}),
 Prediction(uid='76561198032400276', iid=296470, r_ui=2.0, est=2.3660487289938015, details={'actual_k': 45, 'was_impossible': False}),
 Prediction(uid='Subway69', iid=278460, r_ui=0.0, est=0.7122655805816416, details={'actual_k': 45, 'was_impossible': False}),
 Prediction(uid='76561198075973825', iid=200210, r_ui=0.0, est=2.3595363903